In [ ]:
id = 8

In [ ]:
pID = 'sub-' + "%01d" % (id)

import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score, KFold

import pickle, json, os
import scipy.io

from bci_funcs import bandpass_filter_fft, calculate_velocity, gaze_remove_invalid_samples

path = 'P:\\Lukas_Gehrke\\NAH\\data\\5_single-subject-EEG-analysis'

# Explore Behavior

In [ ]:
behavior = pd.read_csv(path+os.sep+pID+os.sep+'behavior_s'+str(id)+'.csv', sep=';')

In [ ]:
behavior = behavior[behavior.bad_epoch != 1]

In [ ]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile) for AccuracyCm
Q1_AccuracyCm = behavior['AccuracyCm'].quantile(0.25)
Q3_AccuracyCm = behavior['AccuracyCm'].quantile(0.75)

# Calculate IQR for AccuracyCm
IQR_AccuracyCm = Q3_AccuracyCm - Q1_AccuracyCm

# Define the lower and upper bounds for outliers in AccuracyCm
lower_bound_AccuracyCm = Q1_AccuracyCm - 1.5 * IQR_AccuracyCm
upper_bound_AccuracyCm = Q3_AccuracyCm + 1.5 * IQR_AccuracyCm

# Calculate Q1 (25th percentile) and Q3 (75th percentile) for fix_delay
Q1_fix_delay = behavior['fix_delay'].quantile(0.25)
Q3_fix_delay = behavior['fix_delay'].quantile(0.75)

# Calculate IQR for fix_delay
IQR_fix_delay = Q3_fix_delay - Q1_fix_delay

# Define the lower and upper bounds for outliers in fix_delay
lower_bound_fix_delay = Q1_fix_delay - 1.5 * IQR_fix_delay
upper_bound_fix_delay = Q3_fix_delay + 1.5 * IQR_fix_delay

# Filter out the outliers
behavior_filt = behavior[
    (behavior['AccuracyCm'] >= lower_bound_AccuracyCm) & (behavior['AccuracyCm'] <= upper_bound_AccuracyCm) &
    (behavior['fix_delay'] >= lower_bound_fix_delay) & (behavior['fix_delay'] <= upper_bound_fix_delay)
]

# print both behavior and behavior_filt shape with some text
print('Behavior shape:', behavior.shape)
print('Behavior_filt shape:', behavior_filt.shape)

behavior = behavior_filt

In [ ]:
with sns.plotting_context('paper', font_scale = 1.8):

    ### Create new plot
    fig, ax = plt.subplots(1, 1, figsize=(7,4))
    fig.patch.set_alpha(1)

    sns.despine() #bottom=True, left=True

    sns.histplot(behavior_filt['answerID']) #, binwidth=1, binrange=(0.5, 5.5), kde=False)

    # add xline at median
    plt.axvline(behavior_filt['answerID'].median(), color='black', linestyle='dashed', linewidth=1)

    plt.xlabel('Answer ID')
    plt.ylabel('Frequency')
    plt.title('Histogram of Answer ID')
    plt.show()

In [ ]:
# Define the order for the hapticProfile categories
haptic_order = sorted(behavior['hapticProfile'].unique())

# Convert the hapticProfile column to a categorical type with the specified order
behavior['hapticProfile'] = pd.Categorical(behavior['hapticProfile'], categories=haptic_order, ordered=True)

fig, axs = plt.subplots(1, 3, figsize=(21, 4))
fig.patch.set_alpha(1)

with sns.plotting_context('paper', font_scale=1.8):
    sns.despine()
    sns.boxplot(x='hapticProfile', y='answerID', data=behavior, ax=axs[0])
    axs[0].set_xlabel('Haptic Profile')
    axs[0].set_ylabel('Answer ID')

with sns.plotting_context('paper', font_scale=1.8):
    sns.despine()
    sns.boxplot(x='hapticProfile', y='AccuracyCm', data=behavior, ax=axs[1])
    axs[1].set_xlabel('Haptic Profile')
    axs[1].set_ylabel('AccuracyCm')

with sns.plotting_context('paper', font_scale=1.8):
    sns.despine()
    sns.boxplot(x='hapticProfile', y='fix_delay', data=behavior, ax=axs[2])
    axs[2].set_xlabel('Haptic Profile')
    axs[2].set_ylabel('Fixation Delay')

plt.tight_layout()
plt.show()


# Neuroadaptive Haptics BCI Training

In [ ]:
# load data
erp = scipy.io.loadmat(path+os.sep+pID+os.sep+'erp.mat')['erp']
gaze = scipy.io.loadmat(path+os.sep+pID+os.sep+'gaze.mat')['gaze']
hand_motion = scipy.io.loadmat(path+os.sep+pID+os.sep+'hand_motion.mat')['hand_motion']

### Signal Processing

In [ ]:
# ERP
# Apply the bandpass filter to the ERP data
erp = bandpass_filter_fft(erp, .1, 15, 250)

# Gaze
gaze_vel = np.zeros((gaze.shape[1], gaze.shape[2]))
# Calculate the velocity for each trial
for i in range(gaze.shape[2]):
    gaze_vel[:, i] = calculate_velocity(gaze[:, :, i])
# remove invalid samples
for i in range(gaze.shape[2]):
    gaze_vel[:, i] = gaze_remove_invalid_samples(gaze_vel[:,i], gaze[-1,:,i])

# Hand motion
hand_vel = np.zeros((hand_motion.shape[1], hand_motion.shape[2]))
# Calculate the velocity for each trial
for i in range(hand_motion.shape[2]):
    hand_vel[:, i] = calculate_velocity(hand_motion[:, :, i])
# add a singleton dim in 1st place for hand_vel
hand_vel = np.expand_dims(hand_vel, axis=0)
# Apply the bandpass filter to the ERP data
hand_vel = bandpass_filter_fft(hand_vel, .1, 15, 250)
# drop singleton dim
hand_vel = np.squeeze(hand_vel, axis=0)

## Select Data and Assign to Classes

In [ ]:
# Simplest approach: select indeces in behavior where answerID is > mean
# this will work with values obtained from a continuous slider in VR where a clear split at the median is possible

high = behavior[behavior['answerID'] > behavior['answerID'].median()].index
low = behavior[behavior['answerID'] < behavior['answerID'].median()].index

## Plots

### EEG

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# erp low
erp_low = erp[:,:,low]
# erp high
erp_high = erp[:,:,high]

plot_erp_low = np.mean(erp_low[4,:,:], axis=1)
plot_erp_high = np.mean(erp_high[4,:,:], axis=1)

erp_baseline_tw = np.arange(250, 262)

# ! fix for final correct baseline correction -> however this is just for plot
plot_erp_low = plot_erp_low - np.mean(plot_erp_low[erp_baseline_tw])
plot_erp_high = plot_erp_high - np.mean(plot_erp_high[erp_baseline_tw])

# merge both conditions
df = pd.DataFrame()
# voltage vector
df['low'] = plot_erp_low
df['high'] = plot_erp_high

# melt
df = pd.melt(df, value_vars=['low', 'high'], var_name='condition', value_name='voltage')

# time vector
df['time'] = np.tile(np.arange(0, len(plot_erp_low)),2)# - 50
df['time'] = (df['time'] * 1/250 * 1000) - 1000

with sns.plotting_context('paper', font_scale = 1.8):

    ### Create new plot
    fig, ax = plt.subplots(1, 1, figsize=(7,4))
    fig.patch.set_alpha(1)

    sns.despine() #bottom=True, left=True

    # sns.lineplot(x='time', y='voltage', hue='condition', style='control', data=df, ax=ax, errorbar='ci')
    sns.lineplot(x='time', y='voltage', hue='condition', data=df, ax=ax, errorbar='ci')
    # sns.lineplot(x='time', y='voltage', hue='control', data=df, ax=ax, errorbar='ci')

    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Amplitude (μV)')

    # xline at 0 with text label 's'
    ax.axvline(0, color='black', linestyle='--')
    ax.text(.1, 4, 'grab', fontsize=14, rotation=90)

    # yline at 0
    ax.axhline(0, color='black', linestyle='--')

    # mark area of feature extraction add a grey background to the y axis range 100 - 300ms
    ax.axvspan(100, 600, color='grey', alpha=0.2, label='features')
    ax.axvspan(0, 50, color='grey', alpha=0.1, label='baseline')

    # move legend to the right
    # ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    # save as eps
    # plt.tight_layout()
    # plt.savefig('/Users/lukasgehrke/Desktop/erp_diff.pdf', format='pdf', dpi=300)

### Motion

In [ ]:
# erp low
erp_low = hand_vel[:,low].mean(axis=1)
# erp high
erp_high = hand_vel[:,high].mean(axis=1)

# merge both conditions
df = pd.DataFrame()
# voltage vector
df['low'] = erp_low
df['high'] = erp_high

# melt
df = pd.melt(df, value_vars=['low', 'high'], var_name='condition', value_name='voltage')

# time vector
df['time'] = np.tile(np.arange(0, len(erp_low)),2)# - 50
df['time'] = (df['time'] * 1/250 * 1000) - 1000

with sns.plotting_context('paper', font_scale = 1.8):

    ### Create new plot
    fig, ax = plt.subplots(1, 1, figsize=(7,4))
    fig.patch.set_alpha(1)

    sns.despine() #bottom=True, left=True

    # sns.lineplot(x='time', y='voltage', hue='condition', style='control', data=df, ax=ax, errorbar='ci')
    sns.lineplot(x='time', y='voltage', hue='condition', data=df, ax=ax, errorbar='ci')
    # sns.lineplot(x='time', y='voltage', hue='control', data=df, ax=ax, errorbar='ci')

    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Amplitude (μV)')

    # xline at 0 with text label 's'
    ax.axvline(0, color='black', linestyle='--')
    # ax.text(-5, .1, 'grab', fontsize=14, rotation=90)

    # yline at 0
    # ax.axhline(0, color='black', linestyle='--')

    # ! mark area of feature extraction add a grey background to the y axis range 100 - 300ms
    # ax.axvspan(50, 450, color='grey', alpha=0.2, label='features')
    # ax.axvspan(0, 50, color='grey', alpha=0.1, label='baseline')

    # move legend to the right
    # ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    # save as eps
    # plt.tight_layout()
    # plt.savefig('/Users/lukasgehrke/Desktop/erp_diff.pdf', format='pdf', dpi=300)

### EYE Tracking

### Fixations

In [ ]:
# fixation_low = np.array(behavior.loc[low, 'fix_delay']).reshape(1, -1)
# fixation_high = np.array(behavior.loc[high, 'fix_delay']).reshape(1, -1)

# data = np.concatenate((fixation_low, fixation_high), axis = 1).T
# low_class = np.zeros((fixation_low.shape[1], 1))
# high_class = np.ones((fixation_high.shape[1], 1))
# classes = np.concatenate((low_class, high_class)).ravel()

# data_to_plot = pd.DataFrame(data)
# data_to_plot['Condition'] = classes
# # rename zeros in Condition to low and ones to high
# data_to_plot['Condition'] = data_to_plot['Condition'].replace({0: 'low', 1: 'high'})
# data_to_plot = data_to_plot.melt(id_vars = 'Condition', value_name = 'amplitude')

# import seaborn as sns
# import matplotlib.pyplot as plt

# with sns.plotting_context('paper', font_scale = 1.8):

#     ### Create new plot
#     fig, ax = plt.subplots(1, 1, figsize=(5,4))
#     fig.patch.set_alpha(1)

#     sns.despine() #bottom=True, left=True

#     sns.histplot(x="amplitude", hue="Condition", data=data_to_plot, kde=True, fill=True, stat="density", legend=True)

#     # xlabel
#     plt.xlabel('Time to first target fixation (ms)')


# Compute Features

In [ ]:
sample_event = 250
data_selection = 144 # 8 windows with 12 samples each
samples_of_int = np.arange(sample_event, sample_event+data_selection)

### ERPs

In [ ]:
erp_selected = erp[:,samples_of_int,:]
erp_selected = erp_selected.reshape(erp_selected.shape[0], 12, 12, erp_selected.shape[2])
erp_selected = np.mean(erp_selected, axis=2)
baseline = erp_selected[:,0,:]
erp_corrected = erp_selected - baseline[:,np.newaxis,:]

erp_low = erp_corrected[:,:,low]
erp_high = erp_corrected[:,:,high]

In [ ]:
erp_low_plot = np.mean(erp_low, axis=2)
erp_high_plot = np.mean(erp_high, axis=2)

# plot them
fig, axs = plt.subplots(2, 4, figsize=(20, 10))
fig.patch.set_alpha(1)

for i, ax in enumerate(axs.flat):
    sns.lineplot(data=erp_low_plot[i], ax=ax, errorbar='sd')
    sns.lineplot(data=erp_high_plot[i], ax=ax, errorbar='sd')
    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Amplitude (μV)')

### Channel Selection

In [ ]:
# Compute the mean ERP for each class
mean_erp_low = np.mean(erp_low, axis=2)
mean_erp_high = np.mean(erp_high, axis=2)

# Subtract the mean ERP of the high class from the mean ERP of the low class and square the result
erp_diff_squared = (mean_erp_low - mean_erp_high) ** 2

# Compute the mean over the 12 windows for each channel
erp_diff_mean = np.mean(erp_diff_squared, axis=1)

# Sort the channels by the absolute differences
sorted_channels = np.argsort(np.abs(erp_diff_mean))[::-1]

# Print the sorted channels
print("Sorted channels by differences:", sorted_channels)

# select the top 10 channels
top_channels = sorted_channels[:10] # needs to be saved for real-time application

# save the top channels
with open(path+os.sep+pID+os.sep+'top_channels.json', 'w') as f:
    json.dump(top_channels.tolist(), f)

### Hand motion

In [ ]:
hand_vel_selected = hand_vel[samples_of_int,:]

# hand_vel_selected = hand_vel_selected.reshape(12, 12, hand_vel_selected.shape[1])
# hand_vel_selected = np.mean(hand_vel_selected, axis=1)
# baseline = hand_vel_selected[:,0,:]
# erp_corrected = hand_vel_selected - baseline[:,np.newaxis,:]

hand_vel_low = hand_vel_selected[:,low]
hand_vel_high = hand_vel_selected[:,high]

In [ ]:
hand_vel_low_plot = np.mean(hand_vel_low, axis=1)
hand_vel_high_plot = np.mean(hand_vel_high, axis=1)

# plot them
fig, ax = plt.subplots(figsize=(10, 6))
fig.patch.set_alpha(1)

sns.lineplot(data=hand_vel_low_plot, ax=ax, errorbar='sd', label='Low')
sns.lineplot(data=hand_vel_high_plot, ax=ax, errorbar='sd', label='High')
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Amplitude (μV)')
ax.legend()
plt.show()

## Eye Tracking

In [ ]:
# EYE - gaze velocity? fixation frequency per second?


# EYE - time to first target fixation
fixation_low = np.array(behavior.loc[low, 'fix_delay']).reshape(1, -1)
fixation_high = np.array(behavior.loc[high, 'fix_delay']).reshape(1, -1)

# Fit Model and Cross-validate

### Feature Vector

In [ ]:
# drop baseline dim
erp_low = erp_low[top_channels,2:,:]
erp_high = erp_high[top_channels,2:,:]
erp_low_flattened = erp_low.reshape(-1, erp_low.shape[-1])
erp_high_flattened = erp_high.reshape(-1, erp_high.shape[-1])

hand_vel_low = np.max(hand_vel_low, axis=0).reshape(1, -1)
hand_vel_high = np.max(hand_vel_high, axis=0).reshape(1, -1)

# concat all low features
#features_low = np.concatenate((erp_low_flattened, fixation_low), axis=0)
#features_low = np.concatenate((erp_low_flattened, hand_vel_low, fixation_low), axis=0)
features_low = erp_low_flattened
# concat all high features
#features_high = np.concatenate((erp_high_flattened, fixation_high), axis=0)
#features_high = np.concatenate((erp_high_flattened, hand_vel_high, fixation_high), axis=0)
features_high = erp_high_flattened

### Train-Test Split

In [ ]:
data = np.concatenate((features_low, features_high), axis = 1).T
low_class = np.zeros((features_low.shape[1], 1))
high_class = np.ones((features_high.shape[1], 1))
classes = np.concatenate((low_class, high_class)).ravel()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data = scaler.fit_transform(data)
pickle.dump(scaler, open(path+os.sep+pID+os.sep+'scaler.sav', 'wb'))

# generate a train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, classes, test_size=0.2, random_state=42)

### Model fitting

In [ ]:
clf = LDA(solver='eigen', shrinkage='auto')

# Perform cross-validation on the training data
cv_scores = cross_val_score(clf, X_train, y_train)
print(f"Mean cross-validation score: {np.mean(cv_scores)}")

# Train the LDA model on the entire training set
clf.fit(X_train, y_train)

# Set boundaries based on LDA scores for real-time application
lda_scores = clf.transform(X_test)
# save lda scores
with open(path+os.sep+pID+os.sep+'lda_scores.json', 'w') as f:
    json.dump(lda_scores.tolist(), f)

# boundary_low = np.percentile(lda_scores, 5)
# # boundary_low = np.min(lda_scores)
# boundary_high = np.percentile(lda_scores, 95)
# # boundary_high = np.max(lda_scores)
# print(f"Boundaries: {boundary_low}, {boundary_high}")

# # save low and high boundary
# with open(path+os.sep+pID+os.sep+'boundaries.json', 'w') as f:
#     json.dump([boundary_low, boundary_high], f)

# save model
clf.fit(data, classes)
filename = path+os.sep+pID+os.sep+'model.sav'
pickle.dump(clf, open(filename, 'wb'))

# save BCI params
target_class = 1
mean_fix_delay = np.mean(behavior['fix_delay'])

# 'f1', 'threshold',
bci_params = dict(((k, eval(k)) for k in ('target_class', 'mean_fix_delay')))
with open(path+os.sep+pID+os.sep+'bci_params.json', 'w') as f:
    json.dump(bci_params, f)
print(bci_params)

In [ ]:
# min max scaling
normalized_lda_scores = (lda_scores - lda_scores.min()) / (lda_scores.max() - lda_scores.min())
# print(f"Boundary scores: {normalized_lda_scores}")

with sns.plotting_context('paper', font_scale = 1.8):

    ### Create new plot
    fig, ax = plt.subplots(1, 1, figsize=(7,4))
    fig.patch.set_alpha(1)

    sns.despine() #bottom=True, left=True

    sns.histplot(normalized_lda_scores, binwidth=0.1, kde=True, ax=ax)
    # xline boundaries
    # ax.axvline(boundary_low, color='black', linestyle='--')
    # ax.axvline(boundary_high, color='black', linestyle='--')
    plt.xlabel('Normalized LDA scores')
    plt.ylabel('Frequency')
    # plt.title('Histogram of decision function scores')
    plt.show()


In [ ]:
# print("Predicted classes: ", pred_classes)
# print("Actual classes:    ", y_test)

# # confusion matrix
# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(y_test, pred_classes)
# print("Confusion matrix: ")
# print(cm)

# # ROC curve
# from sklearn.metrics import roc_curve, roc_auc_score

# fpr, tpr, thresholds = roc_curve(y_test, predictions_prob[:,1])
# roc_auc = roc_auc_score(y_test, predictions_prob[:,1])


# # plot ROC curve
# fig, ax = plt.subplots(figsize=(6, 6))
# fig.patch.set_alpha(1)

# plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], 'k--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.0])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic')
# plt.legend(loc="lower right")
# plt.show()


In [ ]:
# # plot histogram of scores
# data_to_plot = pd.DataFrame(scores)
# data_to_plot['Condition'] = X_test
# # rename zeros in Condition to low and ones to high
# data_to_plot['Condition'] = data_to_plot['Condition'].replace({0: 'low', 1: 'high'})
# data_to_plot = data_to_plot.melt(id_vars = 'Condition', value_name = 'score')

# import seaborn as sns

# with sns.plotting_context('paper', font_scale = 1.8):

#     ### Create new plot
#     fig, ax = plt.subplots(1, 1, figsize=(5,4))
#     fig.patch.set_alpha(1)

#     sns.despine() #bottom=True, left=True

#     sns.histplot(x="score", hue="Condition", data=data_to_plot, kde=True, fill=True, stat="density", legend=True)

#     # add xlines for boundaries variable
#     for boundary in boundaries:
#         ax.axvline(boundary, color='black', linestyle='--')

#     # xlabel
#     plt.xlabel('LDA score')

### ROC interpolated

In [ ]:
# from sklearn.metrics import roc_curve, auc
# import matplotlib.pyplot as plt

# def plot_roc_curve(fpr, tpr):
#     plt.plot(fpr, tpr, color='orange', label='ROC')
#     plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('Receiver Operating Characteristic (ROC) Curve')
#     plt.legend()
#     plt.show()

# def interp_roc_curve(true, score, by ='fp', byval = np.linspace(0,1,101)):
#     fp, tp, th =  roc_curve(true, score)
#     data = {'fp': fp, 'tp': tp, 'th': th}
#     interpData = {}
#     for k,v in data.items():
#         if k==by:
#             interpData[k] = byval
#         else:
#             interpData[k] = np.interp(byval, data[by], data[k])
#     roc = pd.DataFrame.from_dict(interpData)
#     return(roc)

# roc = interp_roc_curve(classes, predictions_prob[:,1])

# false_positive_rate, recall, thresholds = roc_curve(classes, predictions_prob[:,1])

# np.savetxt(path+os.sep+pID+os.sep+'fpr.csv', roc.fp, delimiter=',')
# np.savetxt(path+os.sep+pID+os.sep+'recall.csv', roc.tp, delimiter=',')

# roc_auc = auc(false_positive_rate, recall)
# plot_roc_curve(false_positive_rate, recall)

# optimal_idx = np.argmax(recall - false_positive_rate)
# optimal_threshold = thresholds[optimal_idx]
# print("Threshold value is:", optimal_threshold)

# from sklearn.metrics import f1_score
# f1 = f1_score(classes, pred_classes)
# print("F1 Score: "+ str(f1_score(classes, pred_classes)))

# # threshold = cv_results.mean()
# threshold = np.min(thresholds[false_positive_rate<.2])
# print("Threshold used is:", threshold)